In [29]:
using Plots
using LinearAlgebra
using Random
using Polynomials
using DelimitedFiles;
using Base.Threads

In [30]:
function estica_encolhe(B,C,indice,fator)
        B[:, indice]=(1/fator)*B[:, indice]
        C[indice, :]=fator*C[indice, :]
    return B,C  
end

estica_encolhe (generic function with 1 method)

In [31]:
function troca(B,C,i,j)
    n,=size(B)
    aux = zeros(n)
    aux[:]=B[:, i]
    B[:, i]=B[:, j]
    B[:, j]=aux[:]

    m,y=size(C)
    aux2 = zeros(y)    
    aux2[:]=C[i, :]
    C[i, :]=C[j, :]
    C[j, :]=aux2[:]
     
    return B,C
end

troca (generic function with 1 method)

In [32]:
function soma_subtracao(B,C,i,j,fator)
    B[:, j]+=fator*(B[:, i])
    C[i, :]-=fator*C[j, :]
    return B,C
end

soma_subtracao (generic function with 1 method)

In [33]:
function zerar_coluna_Thread(B,C, i, j, size_linha)
    
    
    Threads.@threads for k in 1:size_linha 
        if (k!=i)
            B,C=soma_subtracao(B,C, k, i, C[k,j])
        end
    end


    return B, C
end    

zerar_coluna_Thread (generic function with 1 method)

In [34]:
function zerar_coluna(B,C, i, j, size_linha)
    
    
 for k in 1:size_linha 
        if (k!=i)
            B,C=soma_subtracao(B,C, k, i, C[k,j])
        end
    end


    return B, C
end    

zerar_coluna (generic function with 1 method)

In [35]:
function encontra_pivo(A, i,j, size_linha)
    i_troca=0
    for k in i:size_linha
        if(A[k,j]!=0.0)
            i_troca=k
            break
        end   
    end
    return i_troca  
end

encontra_pivo (generic function with 1 method)

In [36]:
function Gauss_Jordan(C)
    A=copy(C)
    i_atual=1
    j_atual=1
    
    size_linha, size_coluna=size(A)
    minimo=min(size_linha, size_coluna)
    identidade=zeros(minimo, minimo)
    
    for i in 1:minimo 
        identidade[i,i]=1.0
    end
   
    while (i_atual<=size_linha && j_atual<=size_coluna)
        i_troca = encontra_pivo(A,i_atual, j_atual, size_linha)
        if(i_troca == 0)
            if(i_atual==size_linha)
                j_atual=size_coluna
                continue
            end
            j_atual+=1
            continue
        end
        if(i_atual!=i_troca)
            identidade, A=troca(identidade, A, i_atual, i_troca)
        end
        identidade, A = estica_encolhe(identidade, A, i_atual, 1/A[i_atual, j_atual])
        identidade, A = zerar_coluna(identidade, A, i_atual, j_atual, size_linha)
        i_atual+=1
        j_atual+=1  
    end


    return identidade, A
end    

Gauss_Jordan (generic function with 1 method)

In [37]:
function Gauss_Jordan_T(C)
    A=copy(C)
    i_atual=1
    j_atual=1
    
    size_linha, size_coluna=size(A)
    minimo=min(size_linha, size_coluna)
    identidade=zeros(minimo, minimo)
    
    for i in 1:minimo 
        identidade[i,i]=1.0
    end
   
    while (i_atual<=size_linha && j_atual<=size_coluna)
        i_troca = encontra_pivo(A,i_atual, j_atual, size_linha)
        if(i_troca == 0)
            if(i_atual==size_linha)
                j_atual=size_coluna
                continue
            end
            j_atual+=1
            continue
        end
        if(i_atual!=i_troca)
            identidade, A=troca(identidade, A, i_atual, i_troca)
        end
        identidade, A = estica_encolhe(identidade, A, i_atual, 1/A[i_atual, j_atual])
        identidade, A = zerar_coluna_Thread(identidade, A, i_atual, j_atual, size_linha)
        i_atual+=1
        j_atual+=1  
    end


    return identidade, A
end    

Gauss_Jordan_T (generic function with 1 method)

In [39]:
A = [2.0 4.0 1.0 8.0 11.0 101.0; 
     3.0 4.0 2.0 8.0 12.0 102.0; 
     2.0 4.0 3.0 8.0 13.0 103.0; 
     2.0 4.0 4.0 8.0 14.0 101.0]

A = [1.0 1.0 1.0 1.0 1.0 1.0; 1.0 2.0 3.0 4.0 5.0 6.0; 2.0 3.0 4.0 5.0 6.0 7.0; 0.0 -1.0 -2.0 -3.0 -4.0 -5.0]

A=[1.0 1.0 1.0 2.0 11.0 101.0 1.0 1.0 1.0 2.0 11.0 101.0; 
   1.0 3.0 2.0 4.0 12.0 102.0 1.0 1.0 1.0 2.0 11.0 101.0; 
   1.0 5.0 3.0 6.0 13.0 103.0 1.0 1.0 1.0 2.0 11.0 101.0; 
   1.0 7.0 4.0 8.0 14.0 101.0 1.0 1.0 1.0 2.0 11.0 101.0;
   1.0 1.0 1.0 2.0 11.0 101.0 1.0 1.0 1.0 2.0 11.0 101.0;
   1.0 3.0 2.0 4.0 12.0 102.0 1.0 1.0 1.0 2.0 11.0 101.0; 
   1.0 5.0 3.0 6.0 13.0 103.0 1.0 1.0 1.0 2.0 11.0 101.0; 
   1.0 7.0 4.0 8.0 14.0 101.0 1.0 1.0 1.0 2.0 11.0 101.0 ]


Matriz =[1.0 3.0
         4.0 2.0]

A = round.(rand(10000,10000)) 


@time begin         
B,C = Gauss_Jordan(A)
end

@time begin         
   F,G = Gauss_Jordan_T(A)
   end

println("\n",isapprox(B*C, A))